In [1]:
import pybcclean as pbc
import igl
import numpy as np
import shutil as shut
import wildmeshing as wm
import polyfempy as pf
from scipy.spatial import cKDTree as KDTree
import meshplot as mp
import os
import time
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pbc_utilities as pbcut

In [9]:
obj_root ="../../bcClean_back/data/ABC/obj/abc_0000_obj_v00/"
feat_root = "../../bcClean_back/data/ABC/feat/abc_0000_feat_v00/"
name_offset = 15
count = 0
if not os.path.isdir("../data/test/"):
    os.mkdir("../data/test/")
for parent_suffix in os.listdir(obj_root) :
    obj_parent_dir = obj_root + parent_suffix
    yml_parent_dir = feat_root + parent_suffix
    if (not os.path.isdir(obj_parent_dir)):
        continue
    target_dir = "../data/" + str(count+name_offset)+"/"
    yml_file = pbcut.get_feat_file(yml_parent_dir)
    obj_file = pbcut.get_bad_mesh(obj_parent_dir)
    
    if (yml_file and obj_file):
        file_size = os.path.getsize(obj_file)>> 20 
        # get file size in mb
        if(file_size > 10):
            continue
        if not os.path.isdir(target_dir):
            os.mkdir(target_dir)
        shut.copy2(obj_file, target_dir)
        shut.copy2(yml_file, target_dir)
        print(file_size)
        count +=1
    if (count > 35):
        break
    

2
1
2
4
4
5
2
0
4
2
8
3
10
3
4
4
5
7
3
4
2
4
1
3
4
6
6
6
4
5
4
4
1
4
5
5


In [10]:
for j in range(27,50):
    parent_dir = "../data/"+str(j)+"/"
    obj_file = pbcut.get_bad_mesh(parent_dir)
    yml_file = pbcut.get_feat_file(parent_dir)
    dmat_file = pbcut.parse_feat(yml_file)
    print(dmat_file)
    

../data/15/feat.dmat
../data/16/feat.dmat
../data/17/feat.dmat
../data/18/feat.dmat
../data/19/feat.dmat
../data/20/feat.dmat
../data/21/feat.dmat
../data/22/feat.dmat
../data/23/feat.dmat
../data/24/feat.dmat
../data/25/feat.dmat
../data/26/feat.dmat
../data/27/feat.dmat
../data/28/feat.dmat
../data/29/feat.dmat
../data/30/feat.dmat
../data/31/feat.dmat
../data/32/feat.dmat
../data/33/feat.dmat
../data/34/feat.dmat
../data/35/feat.dmat
../data/36/feat.dmat
../data/37/feat.dmat
../data/38/feat.dmat
../data/39/feat.dmat
../data/40/feat.dmat
../data/41/feat.dmat
../data/42/feat.dmat
../data/43/feat.dmat
../data/44/feat.dmat
../data/45/feat.dmat
../data/46/feat.dmat
../data/47/feat.dmat
../data/48/feat.dmat
../data/49/feat.dmat


In [ ]:
for j in range(28,50):
    parent_dir = "../data/"+str(j)+"/"
    bad_obj_file = pbcut.get_bad_mesh(parent_dir)
    wm.tetrahedralize(bad_obj_file, parent_dir +"good.mesh", stop_quality=7,edge_length_r = 0.04)
    

In [2]:
?wm.tetrahedralize

In [2]:
record_dict={2: {'patch_diff': 0, 'euler_diff': 0}, 3: {'patch_diff': 0, 'euler_diff': 0}, 4: {'patch_diff': -129, 'euler_diff': -47}, 5: {'patch_diff': -19, 'euler_diff': 0}, 6: {'patch_diff': 0, 'euler_diff': 0}, 7: {'patch_diff': 0, 'euler_diff': 0}, 8: {'patch_diff': 0, 'euler_diff': 0}, 9: {'patch_diff': -37, 'euler_diff': -89}, 0: False, 1: False, 10: False, 20: False, 21: {'patch_diff': 0, 'euler_diff': 0}, 22: {'patch_diff': 0, 'euler_diff': 0}, 23: {'patch_diff': 0, 'euler_diff': -6}, 24: {'patch_diff': 0, 'euler_diff': 0}, 25: {'patch_diff': 0, 'euler_diff': -1}, 26: {'patch_diff': 0, 'euler_diff': 0}, 27: {'patch_diff': -27, 'euler_diff': -9}, 28: {'patch_diff': 0, 'euler_diff': -3}, 29: {'patch_diff': 0, 'euler_diff': 0}, 30: {'patch_diff': -1, 'euler_diff': -13}, 31: {'patch_diff': 0, 'euler_diff': 0}, 32: False, 11: {'patch_diff': 0, 'euler_diff': 0}, 12: {'patch_diff': 0, 'euler_diff': 0}, 13: {'patch_diff': 0, 'euler_diff': -8}, 14: {'patch_diff': 0, 'euler_diff': 0}, 15: {'patch_diff': 0, 'euler_diff': 0}, 16: {'patch_diff': 0, 'euler_diff': 0}, 17: {'patch_diff': 0, 'euler_diff': 0}, 18: {'patch_diff': 0, 'euler_diff': 0}, 19: {'patch_diff': 0, 'euler_diff': 0}}
record_dict[32]=False
for j in range(11, 20):
    parent_dir = "../data/"+str(j)+"/"
    bad_obj_file = pbcut.get_bad_mesh(parent_dir)
    dmat_file = parent_dir+"feat.dmat"
    good_obj_file = parent_dir + "good.mesh__sf.obj"
    v_bad, f_bad=igl.read_triangle_mesh(bad_obj_file)
    fl_bad  = igl.read_dmat(dmat_file)
    v_good, f_good = igl.read_triangle_mesh(good_obj_file)
    print("read")
    fl_bad=fl_bad.astype('int32')
    f_good = f_good.astype('int32')
    f_bad = f_bad.astype('int32')
    label_num = fl_bad.max()+1
    prob_mat, fl_good = pbc.project_face_labels(v_bad, f_bad, fl_bad,label_num,2, v_good, f_good)
    fl_refine = pbc.refine_labels(v_good, f_good, prob_mat, fl_good.astype('int32'),1)[:,0]
    print("refine")
    edge_list_bad, patch_edge_list_bad = pbc.build_edge_list(v_bad, f_bad, fl_bad, label_num)
    edge_list_good, patch_edge_list_good = pbc.build_edge_list(v_good, f_good, fl_refine, label_num)
    node_list_bad = pbc.build_node_list(f_bad, fl_bad, label_num)
    node_list_good = pbc.build_node_list(f_good, fl_refine, label_num)
    patch_dict_good = pbcut.extract_patches(v_good, f_good, fl_refine)
    patch_dict_bad = pbcut.extract_patches(v_bad, f_bad, fl_bad)
    pair_edge_dict_bad, adj_matrix_bad = pbcut.build_adj_dict(edge_list_bad, label_num)
    pair_edge_dict_good, adj_matrix_good = pbcut.build_adj_dict(edge_list_good, label_num)
    print("built")
    record_entry = {}
    record_entry["patch_diff"] = len(patch_dict_good) - len(patch_dict_bad)
    euler_good = len(node_list_good)+len(patch_dict_good)-len(edge_list_good)
    euler_bad = len(node_list_bad)+ len(patch_dict_bad)-len(edge_list_bad)
    record_entry["euler_diff"] = euler_good -euler_bad
    record_dict[j]= record_entry
    print(record_dict)
#     except:
#         record_dict[j]= False
            
        

read
refine
built
{2: {'patch_diff': 0, 'euler_diff': 0}, 3: {'patch_diff': 0, 'euler_diff': 0}, 4: {'patch_diff': -129, 'euler_diff': -47}, 5: {'patch_diff': -19, 'euler_diff': 0}, 6: {'patch_diff': 0, 'euler_diff': 0}, 7: {'patch_diff': 0, 'euler_diff': 0}, 8: {'patch_diff': 0, 'euler_diff': 0}, 9: {'patch_diff': -37, 'euler_diff': -89}, 0: False, 1: False, 10: False, 20: False, 21: {'patch_diff': 0, 'euler_diff': 0}, 22: {'patch_diff': 0, 'euler_diff': 0}, 23: {'patch_diff': 0, 'euler_diff': -6}, 24: {'patch_diff': 0, 'euler_diff': 0}, 25: {'patch_diff': 0, 'euler_diff': -1}, 26: {'patch_diff': 0, 'euler_diff': 0}, 27: {'patch_diff': -27, 'euler_diff': -9}, 28: {'patch_diff': 0, 'euler_diff': -3}, 29: {'patch_diff': 0, 'euler_diff': 0}, 30: {'patch_diff': -1, 'euler_diff': -13}, 31: {'patch_diff': 0, 'euler_diff': 0}, 32: False, 11: {'patch_diff': 0, 'euler_diff': 0}, 12: {'patch_diff': 0, 'euler_diff': 0}, 13: {'patch_diff': 0, 'euler_diff': -8}, 14: {'patch_diff': 0, 'euler_diff':

read
refine
built
{2: {'patch_diff': 0, 'euler_diff': 0}, 3: {'patch_diff': 0, 'euler_diff': 0}, 4: {'patch_diff': -129, 'euler_diff': -47}, 5: {'patch_diff': -19, 'euler_diff': 0}, 6: {'patch_diff': 0, 'euler_diff': 0}, 7: {'patch_diff': 0, 'euler_diff': 0}, 8: {'patch_diff': 0, 'euler_diff': 0}, 9: {'patch_diff': -37, 'euler_diff': -89}, 0: False, 1: False, 10: False, 20: False, 21: {'patch_diff': 0, 'euler_diff': 0}, 22: {'patch_diff': 0, 'euler_diff': 0}, 23: {'patch_diff': 0, 'euler_diff': -6}, 24: {'patch_diff': 0, 'euler_diff': 0}, 25: {'patch_diff': 0, 'euler_diff': -1}, 26: {'patch_diff': 0, 'euler_diff': 0}, 27: {'patch_diff': -27, 'euler_diff': -9}, 28: {'patch_diff': 0, 'euler_diff': -3}, 29: {'patch_diff': 0, 'euler_diff': 0}, 30: {'patch_diff': -1, 'euler_diff': -13}, 31: {'patch_diff': 0, 'euler_diff': 0}, 32: False, 11: {'patch_diff': 0, 'euler_diff': 0}, 12: {'patch_diff': 0, 'euler_diff': 0}, 13: {'patch_diff': 0, 'euler_diff': -8}, 14: {'patch_diff': 0, 'euler_diff':

In [12]:
parent_dir = "../data/"+str(25)+"/"
bad_obj_file = pbcut.get_bad_mesh(parent_dir)
dmat_file = parent_dir+"feat.dmat"
good_obj_file = parent_dir + "good.mesh__sf.obj"
v_bad, f_bad=igl.read_triangle_mesh(bad_obj_file)
fl_bad  = igl.read_dmat(dmat_file)
v_good, f_good = igl.read_triangle_mesh(good_obj_file)
print("read")
fl_bad=fl_bad.astype('int32')
f_good = f_good.astype('int32')
f_bad = f_bad.astype('int32')
label_num = fl_bad.max()+1
prob_mat, fl_good = pbc.project_face_labels(v_bad, f_bad, fl_bad,label_num,2, v_good, f_good)
fl_refine = pbc.refine_labels(v_good, f_good, prob_mat, fl_good.astype('int32'),1)[:,0]
print("refine")
edge_list_bad, patch_edge_list_bad = pbc.build_edge_list(v_bad, f_bad, fl_bad, label_num)
edge_list_good, patch_edge_list_good = pbc.build_edge_list(v_good, f_good, fl_refine, label_num)
node_list_bad = pbc.build_node_list(f_bad, fl_bad, label_num)
node_list_good = pbc.build_node_list(f_good, fl_refine, label_num)
patch_dict_good = pbcut.extract_patches(v_good, f_good, fl_refine)
patch_dict_bad = pbcut.extract_patches(v_bad, f_bad, fl_bad)
pair_edge_dict_bad, adj_matrix_bad = pbcut.build_adj_dict(edge_list_bad, label_num)
pair_edge_dict_good, adj_matrix_good = pbcut.build_adj_dict(edge_list_good, label_num)
d = mp.subplot(v_good, f_good, fl_refine, s=[2, 2, 0], shading={"wireframe":True})
diff_mat = (adj_matrix_bad-adj_matrix_good)
area = igl.doublearea(v_good, f_good)
for edg in edge_list_good:
#     if edge_list_good[q].head == -1:
    if diff_mat[edg.label_pair]!=0:
        d.rows[0][0].add_points(v_good[edg.vertices_list], shading={"point_size": 3})
p = mp.subplot(v_bad, f_bad, fl_bad, s= [2,2,0],shading={"wireframe":True})
for edg in edge_list_bad:
#     if edge_list_good[q].head == -1:
    if diff_mat[edg.label_pair]!=0:
        p.rows[0][0].add_points(v_bad[edg.vertices_list], shading={"point_size": 3})


read
refine


In [5]:
dd={2: {'patch_diff': 0, 'euler_diff': 0}, 3: {'patch_diff': 0, 'euler_diff': 0}, 4: {'patch_diff': -129, 'euler_diff': -47}, 5: {'patch_diff': -19, 'euler_diff': 0}, 6: {'patch_diff': 0, 'euler_diff': 0}, 7: {'patch_diff': 0, 'euler_diff': 0}, 8: {'patch_diff': 0, 'euler_diff': 0}, 9: {'patch_diff': -37, 'euler_diff': -89}, 0: False, 1: False, 10: False, 20: False, 21: {'patch_diff': 0, 'euler_diff': 0}, 22: {'patch_diff': 0, 'euler_diff': 0}, 23: {'patch_diff': 0, 'euler_diff': -6}, 24: {'patch_diff': 0, 'euler_diff': 0}, 25: {'patch_diff': 0, 'euler_diff': -1}, 26: {'patch_diff': 0, 'euler_diff': 0}, 27: {'patch_diff': -27, 'euler_diff': -9}, 28: {'patch_diff': 0, 'euler_diff': -3}, 29: {'patch_diff': 0, 'euler_diff': 0}, 30: {'patch_diff': -1, 'euler_diff': -13}, 31: {'patch_diff': 0, 'euler_diff': 0}, 32: False, 11: {'patch_diff': 0, 'euler_diff': 0}, 12: {'patch_diff': 0, 'euler_diff': 0}, 13: {'patch_diff': 0, 'euler_diff': -8}, 14: {'patch_diff': 0, 'euler_diff': 0}, 15: {'patch_diff': 0, 'euler_diff': 0}, 16: {'patch_diff': 0, 'euler_diff': 0}, 17: {'patch_diff': 0, 'euler_diff': 0}, 18: {'patch_diff': 0, 'euler_diff': 0}, 19: {'patch_diff': 0, 'euler_diff': 0}}

In [6]:
for key in sorted(dd.keys()):
    print(key, dd[key])
    

0 False
1 False
2 {'patch_diff': 0, 'euler_diff': 0}
3 {'patch_diff': 0, 'euler_diff': 0}
4 {'patch_diff': -129, 'euler_diff': -47}
5 {'patch_diff': -19, 'euler_diff': 0}
6 {'patch_diff': 0, 'euler_diff': 0}
7 {'patch_diff': 0, 'euler_diff': 0}
8 {'patch_diff': 0, 'euler_diff': 0}
9 {'patch_diff': -37, 'euler_diff': -89}
10 False
11 {'patch_diff': 0, 'euler_diff': 0}
12 {'patch_diff': 0, 'euler_diff': 0}
13 {'patch_diff': 0, 'euler_diff': -8}
14 {'patch_diff': 0, 'euler_diff': 0}
15 {'patch_diff': 0, 'euler_diff': 0}
16 {'patch_diff': 0, 'euler_diff': 0}
17 {'patch_diff': 0, 'euler_diff': 0}
18 {'patch_diff': 0, 'euler_diff': 0}
19 {'patch_diff': 0, 'euler_diff': 0}
20 False
21 {'patch_diff': 0, 'euler_diff': 0}
22 {'patch_diff': 0, 'euler_diff': 0}
23 {'patch_diff': 0, 'euler_diff': -6}
24 {'patch_diff': 0, 'euler_diff': 0}
25 {'patch_diff': 0, 'euler_diff': -1}
26 {'patch_diff': 0, 'euler_diff': 0}
27 {'patch_diff': -27, 'euler_diff': -9}
28 {'patch_diff': 0, 'euler_diff': -3}
29 {'p

In [53]:
flag_bad,vv_bad,tv_bad,fv_bad =igl.tetrahedralize(v_bad, f_bad)

In [54]:
print(flag_bad)

0


In [55]:
igl.

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [56]:

def barycenters(v, f):
    s = np.zeros((len(f),3),dtype='float64')
    for i, r in enumerate(s):
        s[i] = (v[f[i][0]]+v[f[i][1]]+v[f[i][2]])/3
    return s
bary_centers_good = barycenters(v_good, f_good)
kdt_good = KDTree(bary_centers_good)

bary_centers_bad = barycenters(v_bad, f_bad)
kdt_bad = KDTree(bary_centers_bad)

def sideset_good(p):
    return fl_refine[kdt_good.query(p)[1]] + 1
def sideset_bad(p):
    res =fl_bad[kdt_bad.query(p)[1]]+ 1
    return fl_bad[kdt_bad.query(p)[1]]+ 1



In [57]:
settings = pf.Settings()
problem = pf.Problem()
solver_bad = pf.Solver()
solver_good = pf.Solver()
settings.normalize_mesh = False
settings.set_pde(pf.PDEs.Laplacian)
problem.add_dirichlet_value(6,1.0)
problem.add_dirichlet_value(7,-1.0)
settings.set_problem(problem)
solver_bad.settings(settings)
solver_good.settings(settings)
solver_bad.set_mesh(vv_bad, tv_bad)
solver_bad.set_boundary_side_set_from_bary(sideset_bad)
solver_good.load_mesh_from_path(parent_dir+"good.mesh")
solver_good.set_boundary_side_set_from_bary(sideset_good)
b_bad, tt_bad, ss_bad = solver_bad.get_boundary_sidesets()
mp.plot(b_bad, tt_bad, ss_bad, shading={"wireframe":True})
print(ss_bad.max())

[2019-09-26 13:38:26.407] [polyfem] [info] Loading mesh...
[2019-09-26 13:38:26.407] [polyfem] [info] mesh bb min [-60.9219, -60.9217, -67.31], max [60.9219, 60.9217, 0]
[2019-09-26 13:38:26.411] [polyfem] [info]  took 0.00333186s
[2019-09-26 13:38:29.914] [polyfem] [info] Loading mesh...
[2019-09-26 13:38:29.914] [geogram] [info] Loading file ../data/13/good.mesh...
[2019-09-26 13:38:29.952] [geogram] [info] (FP64) nb_v:2601 nb_e:0 nb_f:3678 nb_b:0 tri:1 dim:3
[2019-09-26 13:38:29.952] [geogram] [info]  nb_tets:9540
[2019-09-26 13:38:29.952] [geogram] [info] Attributes on vertices: point[3]
[2019-09-26 13:38:30.069] [polyfem] [info] mesh bb min [-60.9184, -60.9205, -67.31], max [60.9009, 60.9201, 2.22045e-16]
[2019-09-26 13:38:30.070] [polyfem] [info]  took 0.155292s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

15.0


In [58]:

solver_bad.solve()

solver_good.solve()

[2019-09-26 13:38:39.440] [polyfem] [info] simplex_count: 	51865
[2019-09-26 13:38:39.440] [polyfem] [info] regular_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] regular_boundary_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] simple_singular_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] multi_singular_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] boundary_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] multi_singular_boundary_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] non_regular_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] non_regular_boundary_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] undefined_count: 	0
[2019-09-26 13:38:39.440] [polyfem] [info] total count:	 51865
[2019-09-26 13:38:39.441] [polyfem] [info] Building isoparametric basis...
[2019-09-26 13:38:39.684] [polyfem] [info] Computing polygonal basis...
[2019-09-26 13:38:39.684] [polyfem] [info]  took 2.3871e-05s
[2019-09-26 13:38:39.699] [polyfem] [info] hmin: 0.0657

In [59]:
p_bad, t_bad, d_bad = solver_bad.get_sampled_solution(boundary_only=False)
mp.plot(p_bad, t_bad, d_bad[:,0], shading={'flat':True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [60]:
p_good, t_good, d_good = solver_good.get_sampled_solution(boundary_only=True)
mp.plot(p_good, t_good, d_good[:,0], shading={'flat':True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.008790…